In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Start

In this guide, we'll show how you can use EvalML to automatically find the best pipeline for predicting whether a patient has breast cancer. Along the way, we'll highlight EvalML's built-in tools and features for understanding and interacting with the search process.

In [ ]:
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types

First, we load in the features and outcomes we want to use to train our model.

In [ ]:
X, y = evalml.demos.load_fraud(n_rows=1000, return_pandas=True)

First, we will clean the data. Since EvalML accepts a pandas input, it can run type inference on this data directly. Since we'd like to change the types inferred by EvalML, we can use the `infer_feature_types` utility method. Here's what we're going to do with the following dataset:

- Reformat the `expiration_date` column so it reflects a more familiar date format.
- Cast the `lat` and `lng` columns from float to str.
- Use `infer_feature_types` to specify what types certain columns should be. For example, to avoid having the `provider` column be inferred as natural language text, we have specified it as a categorical column instead.

The `infer_feature_types` utility method takes a pandas or numpy input and converts it to a [Woodwork](https://woodwork.alteryx.com/en/stable/) data structure, providing us with flexibility to cast the data as necessary. 

In [ ]:
X['expiration_date'] = X['expiration_date'].apply(lambda x: '20{}-01-{}'.format(x.split("/")[1], x.split("/")[0]))
X[['lat', 'lng']] = X[['lat', 'lng']].astype('str')
X = infer_feature_types(X, feature_types= {'store_id': 'categorical',
                                           'expiration_date': 'datetime', 
                                           'lat': 'categorical',
                                           'lng': 'categorical',
                                           'provider': 'categorical'})
X

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set.

In [ ]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=.2)

__Note:__ To provide data to EvalML, it is recommended that you create a `DataTable` object using [the Woodwork project](https://woodwork.alteryx.com/en/stable/). Here, `split_data()` returns Woodwork data structures.

EvalML also accepts and works well with pandas `DataFrames`. But using the `DataTable` makes it easy to control how EvalML will treat each feature, as a numeric feature, a categorical feature, a text feature or other type of feature. Woodwork's `DataTable` includes features like inferring when a categorical feature should be treated as a text feature.

EvalML has many options to configure the pipeline search. At the minimum, we need to define an objective function. For simplicity, we will use the F1 score in this example. However, the real power of EvalML is in using domain-specific [objective functions](user_guide/objectives.ipynb) or [building your own](user_guide/objectives.ipynb).

Below EvalML utilizes Bayesian optimization (EvalML's default optimizer) to search and find the best pipeline defined by the given objective.

EvalML provides a number of parameters to control the search process. `max_batches` is one of the parameters which controls the stopping criterion for the AutoML search. It indicates the maximum number of rounds of AutoML to evaluate, where each round may train and score a variable number of pipelines. In this example, `max_batches` is set to 1.

** Graphing methods, like AutoMLSearch, on Jupyter Notebook and Jupyter Lab require [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html) to be installed.

** If graphing on Jupyter Lab, [jupyterlab-plotly](https://plotly.com/python/getting-started/#jupyterlab-support-python-35) required. To download this, make sure you have [npm](https://nodejs.org/en/download/) installed.

In [ ]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary', objective='f1', max_batches=1)

When we call `search()`, the search for the best pipeline will begin. There is no need to wrangle with missing data or categorical variables as EvalML includes various preprocessing steps (like imputation, one-hot encoding, feature selection) to ensure you're getting the best results. As long as your data is in a single table, EvalML can handle it. If not, you can reduce your data to a single table by utilizing [Featuretools](https://featuretools.featurelabs.com) and its Entity Sets.

You can find more information on pipeline components and how to integrate your own custom pipelines into EvalML [here](user_guide/pipelines.ipynb).

In [ ]:
automl.search()

After the search is finished we can view all of the pipelines searched, ranked by score. Internally, EvalML performs cross validation to score the pipelines. If it notices a high variance across cross validation folds, it will warn you. EvalML also provides additional [data checks](user_guide/data_checks.ipynb) to analyze your data to assist you in producing the best performing pipeline. 

In [ ]:
automl.rankings

If we are interested in see more details about the pipeline, we can view a summary description using the `id` from the rankings table:

In [ ]:
automl.describe_pipeline(3)

We can also view the pipeline parameters directly:

In [ ]:
pipeline = automl.get_pipeline(3)
print(pipeline.parameters)

We can now select the best pipeline and score it on our holdout data:

In [ ]:
pipeline = automl.best_pipeline
pipeline.score(X_holdout, y_holdout, ["f1"])

We can also visualize the structure of the components contained by the pipeline:

In [ ]:
pipeline.graph()